In [2]:
! uv pip install langchain openai tiktoken rapidocr-onnxruntime python-dotenv langchain-community

Using Python 3.12.12 environment at: c:\utube project\llmops-by-sunny-saviya\.venv
Resolved 75 packages in 470ms
 Downloaded shapely
 Downloaded sqlalchemy
 Downloaded pillow
 Downloaded numpy
 Downloaded onnxruntime
Prepared 16 packages in 3.70s
Installed 72 packages in 1.45s
 + aiohappyeyeballs==2.6.1
 + aiohttp==3.13.2
 + aiosignal==1.4.0
 + annotated-types==0.7.0
 + anyio==4.12.0
 + attrs==25.4.0
 + certifi==2025.11.12
 + charset-normalizer==3.4.4
 + coloredlogs==15.0.1
 + dataclasses-json==0.6.7
 + distro==1.9.0
 + flatbuffers==25.12.19
 + frozenlist==1.8.0
 + greenlet==3.3.0
 + h11==0.16.0
 + httpcore==1.0.9
 + httpx==0.28.1
 + httpx-sse==0.4.3
 + humanfriendly==10.0
 + idna==3.11
 + jiter==0.12.0
 + jsonpatch==1.33
 + jsonpointer==3.0.0
 + langchain==1.2.0
 + langchain-classic==1.0.1
 + langchain-community==0.4.1
 + langchain-core==1.2.5
 + langchain-text-splitters==1.1.0
 + langgraph==1.0.5
 + langgraph-checkpoint==3.0.1
 + langgraph-prebuilt==1.0.5
 + langgraph-sdk==0.3.1
 + l

In [4]:
import os

from dotenv import load_dotenv

load_dotenv()


os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")


## Data Ingestion


In [4]:
from langchain.document_loaders import TextLoader

In [6]:
loader = TextLoader("/Users/yashpatil/Developer/AI/YT/Sunny/LLMOps_series/data/Agentic AI.txt", encoding="utf8")
documents = loader.load()

In [8]:
documents[0].page_content[:500]  # Print the first 500 characters of the first documen

'\ufeffUnderstanding Agentic AI\nAgentic AI refers to a new paradigm in artificial intelligence where systems are designed not just to respond to queries or perform specific tasks, but to operate autonomously towards achieving predefined goals. This involves capabilities such as planning, reasoning, executing actions, and continuously adapting to dynamic environments without constant human intervention.\nKey Characteristics of Agentic AI\nAgentic AI systems are distinct from traditional AI models due to '

In [9]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [10]:
text_splitter=RecursiveCharacterTextSplitter(chunk_size=200,chunk_overlap=20)

In [11]:
text_chunks=text_splitter.split_documents(documents)

In [13]:
text_chunks

[Document(metadata={'source': '/Users/yashpatil/Developer/AI/YT/Sunny/LLMOps_series/data/Agentic AI.txt'}, page_content='\ufeffUnderstanding Agentic AI'),
 Document(metadata={'source': '/Users/yashpatil/Developer/AI/YT/Sunny/LLMOps_series/data/Agentic AI.txt'}, page_content='Agentic AI refers to a new paradigm in artificial intelligence where systems are designed not just to respond to queries or perform specific tasks, but to operate autonomously towards achieving'),
 Document(metadata={'source': '/Users/yashpatil/Developer/AI/YT/Sunny/LLMOps_series/data/Agentic AI.txt'}, page_content='towards achieving predefined goals. This involves capabilities such as planning, reasoning, executing actions, and continuously adapting to dynamic environments without constant human intervention.'),
 Document(metadata={'source': '/Users/yashpatil/Developer/AI/YT/Sunny/LLMOps_series/data/Agentic AI.txt'}, page_content='Key Characteristics of Agentic AI\nAgentic AI systems are distinct from traditional 

In [15]:
! uv pip install faiss-cpu


Using Python 3.12.10 environment at: /Users/yashpatil/Developer/AI/YT/Sunny/LLMOps_series/.venv
Resolved 3 packages in 333ms                                         
Installed 1 package in 3ms                                  
 + faiss-cpu==1.12.0


In [16]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS

In [17]:
embeddings=OpenAIEmbeddings()

/var/folders/d_/rqsk7t093dlg0nzx8rc5n4fh0000gn/T/ipykernel_6665/2276573386.py:1: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings=OpenAIEmbeddings()


In [18]:
vectorstore=FAISS.from_documents(text_chunks, embeddings)

In [19]:
vectorstore

In [29]:
retriever=vectorstore.as_retriever()

In [20]:
# Perform similarity search
query = "What is the Key Characteristics of Agentic AI?"
docs = vectorstore.similarity_search(query, k=4)

# Display the results
for i, doc in enumerate(docs):
    print(f"Document {i+1}:")
    print(doc.page_content)
    print("-" * 50)


Document 1:
Key Characteristics of Agentic AI
Agentic AI systems are distinct from traditional AI models due to several core characteristics:
--------------------------------------------------
Document 2:
﻿Understanding Agentic AI
--------------------------------------------------
Document 3:
* Adaptability and Learning: They can adjust their behavior and improve their performance based on new information and experiences.
How Agentic AI Works
--------------------------------------------------
Document 4:
such as ensuring safety, interpretability, and ethical decision-making remain critical areas of research and development for the widespread adoption of agentic AI.
--------------------------------------------------


In [21]:
from langchain.prompts import ChatPromptTemplate

template="""You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Use ten sentences maximum and keep the answer concise.
Question: {question}
Context: {context}
Answer:
"""

In [22]:
prompt=ChatPromptTemplate.from_template(template)

In [23]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks.\nUse the following pieces of retrieved context to answer the question.\nIf you don't know the answer, just say that you don't know.\nUse ten sentences maximum and keep the answer concise.\nQuestion: {question}\nContext: {context}\nAnswer:\n"), additional_kwargs={})])

In [24]:
from langchain.schema.output_parser import StrOutputParser

In [25]:
output_parser=StrOutputParser()

In [26]:
from langchain.chat_models import ChatOpenAI

llm_model=ChatOpenAI(model_name="gpt-4o-mini")

/var/folders/d_/rqsk7t093dlg0nzx8rc5n4fh0000gn/T/ipykernel_6665/2671044564.py:3: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm_model=ChatOpenAI(model_name="gpt-4o-mini")


In [30]:
from langchain.schema.runnable import RunnablePassthrough


rag_chain = (
    {"context": retriever,  "question": RunnablePassthrough()}
    | prompt
    | llm_model
    | output_parser
)

In [31]:
rag_chain.invoke("tell me about Agentic AI")

'Agentic AI represents a new paradigm in artificial intelligence where systems are designed to operate autonomously rather than merely responding to queries or performing specific tasks. These systems exhibit several key characteristics that differentiate them from traditional AI models. One of the main features is their ability to make independent decisions to achieve set goals. The potential applications of agentic AI are extensive and cover various industries, including robotics, where autonomous robots can perform complex tasks in manufacturing, exploration, or logistics. Overall, agentic AI aims to enhance the capabilities of AI systems, allowing them to function more independently and effectively in diverse environments.'

In [33]:
import structlog

ModuleNotFoundError: No module named 'structlog'